In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import os
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn import metrics
from imgaug import augmenters as iaa
import imgaug as ia
from random import shuffle
import cv2
import numpy as np

from DataService import DataService
from EnvironmentSettings import EnvironmentSettings

import matplotlib.pyplot as plt

In [2]:
ENV_SETTINGS = EnvironmentSettings("AZURE")

In [3]:
train_files = glob(ENV_SETTINGS.train + '/*.' + ENV_SETTINGS.image_extension)
print("labeled files size :", len(train_files))

df_train = pd.read_csv(ENV_SETTINGS.labels)
df_train.head()
FILE_LABELS = {k:v for k,v in zip(df_train.id.values, df_train.label.values)}

TRAIN_SET, TEST_SET = train_test_split(train_files, test_size = 0.1, random_state = 42)
print("training size :", len(TRAIN_SET))
print("testing size :", len(TEST_SET))

labeled files size : 220025
training size : 198022
testing size : 22003


In [4]:
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def get_id_from_file_path(file_path):
    #return os.path.basename(file_path)
    return file_path.split(os.path.sep)[-1].replace('.' + ENV_SETTINGS.image_extension, '')

def data_gen(list_files, id_label_map, batch_size):
    while True:
        #shuffle(list_files)
        for batch in chunker(list_files, batch_size):
            X = [cv2.imread(x) for x in batch]
            Y = [id_label_map[get_id_from_file_path(x)] for x in batch]

            yield np.array(X), np.array(Y)


In [5]:
def show_final_history(history):
    fig, ax = plt.subplots(2, 2, figsize=(15,5))
    ax[0][0].set_title('loss')
    ax[0][0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0][0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[0][1].set_title('accuracy')
    ax[0][1].plot(history.epoch, history.history["accuracy"], label="Train acc")
    ax[0][1].plot(history.epoch, history.history["val_accuracy"], label="Validation acc")
    ax[1][0].set_title('F1')
    ax[1][0].plot(history.epoch, history.history["get_f1"], label="Train F1")
    ax[1][0].plot(history.epoch, history.history["val_get_f1"], label="Validation F1")
    ax[1][1].set_title('Auc')
    ax[1][1].plot(history.epoch, history.history["get_auc"], label="Train Auc")
    ax[1][1].plot(history.epoch, history.history["val_get_auc"], label="Validation Auc")
    ax[0][0].legend()
    ax[0][1].legend()
    ax[1][0].legend()
    ax[1][1].legend()
    plt.show()

In [6]:
from tensorflow.keras.callbacks import Callback,ModelCheckpoint
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow.keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val


from tensorflow.keras.callbacks import Callback
from sklearn.metrics import roc_auc_score

# AUC for a binary classifier (https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/discussion/41015)
def get_auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)
    
#---------------------
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)
    return FP/N

#----------------
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)
    return TP/P
    

In [7]:
HEIGHT = WIDTH = 96
CHANNEL = 3
BATCH_SIZE = 128
EPOCHS = 1000

In [8]:
from kerastuner import HyperModel

In [9]:
class CNNHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = Sequential()
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'conv_1',
                    values=[16, 32, 64, 128],
                    default=64,
                ),
                kernel_size=3,
                activation='relu',
                input_shape=self.input_shape
            )
        )
        model.add(MaxPooling2D())
        model.add(Flatten())
        model.add(Dense(1))

        model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
        
        return model

hypermodel = CNNHyperModel(input_shape=(HEIGHT, WIDTH, CHANNEL))

In [14]:
from kerastuner import RandomSearch

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=100,
    executions_per_trial=10,
    directory='my_dir4',
    project_name='hyperparameter-tuning4')

In [15]:
tuner.search_space_summary()

In [16]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
tuner.search(
    data_gen(TRAIN_SET[:51200], FILE_LABELS, BATCH_SIZE), 
    steps_per_epoch=len(TRAIN_SET[:51200]) // BATCH_SIZE,
    epochs=15, 
    validation_data=data_gen(TEST_SET, FILE_LABELS, BATCH_SIZE),
    validation_steps=len(TEST_SET) // BATCH_SIZE,
    callbacks=[early_stopping_callback],
    verbose=2
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 23s - loss: 14.9945 - accuracy: 0.6516 - val_loss: 1.5360 - val_accuracy: 0.6703
Epoch 2/15
400/400 - 23s - loss: 0.9980 - accuracy: 0.6376 - val_loss: 0.7913 - val_accuracy: 0.5961
Epoch 3/15
400/400 - 22s - loss: 0.6486 - accuracy: 0.6247 - val_loss: 0.7032 - val_accuracy: 0.5887
Epoch 4/15
400/400 - 22s - loss: 0.5871 - accuracy: 0.6355 - val_loss: 0.6940 - val_accuracy: 0.5877
Epoch 5/15
400/400 - 22s - loss: 0.5619 - accuracy: 0.6483 - val_loss: 0.6988 - val_accuracy: 0.5884
Epoch 6/15
400/400 - 21s - loss: 0.5446 - accuracy: 0.6613 - val_loss: 0.7084 - val_accuracy: 0.5909
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 22s - loss: 17.3615 - accuracy: 0.6125 - val_loss: 1.3943 - val_accuracy: 0.6137
Epoch 2/15
400/400 - 22s - loss: 0.9546 - accuracy: 0.6217 - val_loss: 0.8622 - val_accuracy: 0.5936
Ep

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 23s - loss: 28.9393 - accuracy: 0.5895 - val_loss: 0.6323 - val_accuracy: 0.5914
Epoch 2/15
400/400 - 22s - loss: 0.6076 - accuracy: 0.5975 - val_loss: 0.6335 - val_accuracy: 0.5921
Epoch 3/15
400/400 - 22s - loss: 0.5633 - accuracy: 0.6212 - val_loss: 0.6572 - val_accuracy: 0.5922
Epoch 4/15
400/400 - 22s - loss: 0.5052 - accuracy: 0.6759 - val_loss: 0.7040 - val_accuracy: 0.5961
Epoch 5/15
400/400 - 22s - loss: 0.4503 - accuracy: 0.7505 - val_loss: 0.8046 - val_accuracy: 0.6359
Epoch 6/15
400/400 - 22s - loss: 0.4202 - accuracy: 0.7970 - val_loss: 0.8813 - val_accuracy: 0.6486
Epoch 7/15
400/400 - 22s - loss: 0.4002 - accuracy: 0.8088 - val_loss: 1.0917 - val_accuracy: 0.6171
Epoch 8/15
400/400 - 22s - loss: 0.4128 - accuracy: 0.7993 - val_loss: 0.9601 - val_accuracy: 0.6352
Epoch 9/15
400/400 - 22s - loss: 0.4013 - accuracy: 0.7979 - val_loss: 1.2676 - val_accuracy: 0.6

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 22s - loss: 18.7067 - accuracy: 0.6092 - val_loss: 0.7301 - val_accuracy: 0.6257
Epoch 2/15
400/400 - 21s - loss: 0.5864 - accuracy: 0.6993 - val_loss: 0.6844 - val_accuracy: 0.6790
Epoch 3/15
400/400 - 21s - loss: 0.4610 - accuracy: 0.7702 - val_loss: 0.7186 - val_accuracy: 0.6833
Epoch 4/15
400/400 - 21s - loss: 0.3747 - accuracy: 0.8255 - val_loss: 0.7908 - val_accuracy: 0.6817
Epoch 5/15
400/400 - 22s - loss: 0.3293 - accuracy: 0.8546 - val_loss: 0.9318 - val_accuracy: 0.6728
Epoch 6/15
400/400 - 21s - loss: 0.3686 - accuracy: 0.8358 - val_loss: 1.0395 - val_accuracy: 0.6772
Epoch 7/15
400/400 - 21s - loss: 0.4027 - accuracy: 0.8169 - val_loss: 1.1753 - val_accuracy: 0.6491
Epoch 8/15
400/400 - 21s - loss: 0.4051 - accuracy: 0.7972 - val_loss: 1.2030 - val_accuracy: 0.6047
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epo

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 23s - loss: 25.1361 - accuracy: 0.6124 - val_loss: 0.8617 - val_accuracy: 0.6377
Epoch 2/15
400/400 - 22s - loss: 0.7239 - accuracy: 0.6568 - val_loss: 0.7601 - val_accuracy: 0.6335
Epoch 3/15
400/400 - 22s - loss: 0.5871 - accuracy: 0.6917 - val_loss: 0.7549 - val_accuracy: 0.6456
Epoch 4/15
400/400 - 22s - loss: 0.5065 - accuracy: 0.7370 - val_loss: 0.8024 - val_accuracy: 0.6533
Epoch 5/15
400/400 - 22s - loss: 0.4543 - accuracy: 0.7704 - val_loss: 0.8642 - val_accuracy: 0.6567
Epoch 6/15
400/400 - 22s - loss: 0.4196 - accuracy: 0.7925 - val_loss: 0.9490 - val_accuracy: 0.6575
Epoch 7/15
400/400 - 22s - loss: 0.4184 - accuracy: 0.7954 - val_loss: 1.0278 - val_accuracy: 0.6548
Epoch 8/15
400/400 - 22s - loss: 0.4525 - accuracy: 0.7779 - val_loss: 1.1555 - val_accuracy: 0.6165
Epoch 9/15
400/400 - 22s - loss: 0.4425 - accuracy: 0.7835 - val_loss: 1.2468 - val_accuracy: 0.6

INFO:tensorflow:Oracle triggered exit


In [17]:
tuner.results_summary()

In [8]:
from kerastuner import HyperModel
class CNNHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = Sequential()
        model.add(
            Conv2D(
                filters=hp.Choice(
                    'conv_filter_1',
                    values=[16, 32, 64, 128],
                    default=64,
                ),
                kernel_size=hp.Choice(
                    'conv_kernel_1',
                    values=[3, 5, 7, 9, 11],
                    default=3,
                ),
                activation='relu',
                padding='same',
                input_shape=self.input_shape
            )
        )
        model.add(MaxPooling2D())
        model.add(Flatten())
        model.add(Dense(1))

        model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
        
        return model

hypermodel = CNNHyperModel(input_shape=(HEIGHT, WIDTH, CHANNEL))

In [9]:
from kerastuner import RandomSearch
tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=100,
    executions_per_trial=6,
    directory='my_dir10',
    project_name='hyperparameter-10')

In [10]:
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4)
tuner.search(
    data_gen(TRAIN_SET[:51200], FILE_LABELS, BATCH_SIZE), 
    steps_per_epoch=len(TRAIN_SET[:51200]) // BATCH_SIZE,
    epochs=15, 
    validation_data=data_gen(TEST_SET, FILE_LABELS, BATCH_SIZE),
    validation_steps=len(TEST_SET) // BATCH_SIZE,
    callbacks=[early_stopping_callback],
    verbose=2
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 1133s - loss: 14.6885 - accuracy: 0.6374 - val_loss: 0.5505 - val_accuracy: 0.7042
Epoch 2/15
400/400 - 32s - loss: 0.5285 - accuracy: 0.7208 - val_loss: 0.5371 - val_accuracy: 0.7157
Epoch 3/15
400/400 - 33s - loss: 0.5068 - accuracy: 0.7361 - val_loss: 0.5220 - val_accuracy: 0.7445
Epoch 4/15
400/400 - 33s - loss: 0.4942 - accuracy: 0.7482 - val_loss: 0.5260 - val_accuracy: 0.7457
Epoch 5/15
400/400 - 32s - loss: 0.4810 - accuracy: 0.7548 - val_loss: 0.5387 - val_accuracy: 0.7509
Epoch 6/15
400/400 - 32s - loss: 0.4759 - accuracy: 0.7591 - val_loss: 0.5632 - val_accuracy: 0.7031
Epoch 7/15
400/400 - 32s - loss: 0.4751 - accuracy: 0.7613 - val_loss: 0.5356 - val_accuracy: 0.7455
Epoch 8/15
400/400 - 32s - loss: 0.4576 - accuracy: 0.7701 - val_loss: 0.5327 - val_accuracy: 0.7574
Epoch 9/15
400/400 - 32s - loss: 0.4519 - accuracy: 0.7746 - val_loss: 0.5597 - val_accuracy: 0

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 55s - loss: 19.5752 - accuracy: 0.6186 - val_loss: 0.5795 - val_accuracy: 0.6890
Epoch 2/15
400/400 - 54s - loss: 0.5171 - accuracy: 0.7429 - val_loss: 0.5155 - val_accuracy: 0.7477
Epoch 3/15
400/400 - 54s - loss: 0.5005 - accuracy: 0.7539 - val_loss: 0.5112 - val_accuracy: 0.7569
Epoch 4/15
400/400 - 54s - loss: 0.4916 - accuracy: 0.7603 - val_loss: 0.5105 - val_accuracy: 0.7611
Epoch 5/15
400/400 - 53s - loss: 0.4862 - accuracy: 0.7645 - val_loss: 0.5074 - val_accuracy: 0.7601
Epoch 6/15
400/400 - 53s - loss: 0.4874 - accuracy: 0.7626 - val_loss: 0.5021 - val_accuracy: 0.7593
Epoch 7/15
400/400 - 54s - loss: 0.4814 - accuracy: 0.7661 - val_loss: 0.5039 - val_accuracy: 0.7618
Epoch 8/15
400/400 - 54s - loss: 0.4705 - accuracy: 0.7717 - val_loss: 0.5011 - val_accuracy: 0.7636
Epoch 9/15
400/400 - 53s - loss: 0.4668 - accuracy: 0.7736 - val_loss: 0.5187 - val_accuracy: 0.7

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 68s - loss: 15.3463 - accuracy: 0.5936 - val_loss: 0.6391 - val_accuracy: 0.5918
Epoch 2/15
400/400 - 66s - loss: 0.6409 - accuracy: 0.5947 - val_loss: 0.6382 - val_accuracy: 0.5915
Epoch 3/15
400/400 - 66s - loss: 0.6352 - accuracy: 0.5950 - val_loss: 0.6398 - val_accuracy: 0.5903
Epoch 4/15
400/400 - 67s - loss: 0.6381 - accuracy: 0.5952 - val_loss: 0.6546 - val_accuracy: 0.5878
Epoch 5/15
400/400 - 66s - loss: 0.6401 - accuracy: 0.5957 - val_loss: 0.6676 - val_accuracy: 0.5898
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 67s - loss: 18.9856 - accuracy: 0.5913 - val_loss: 0.6416 - val_accuracy: 0.5916
Epoch 2/15
400/400 - 67s - loss: 0.5871 - accuracy: 0.6405 - val_loss: 0.5978 - val_accuracy: 0.6731
Epoch 3/15
400/400 - 66s - loss: 0.5929 - accuracy: 0.5946 - val_loss: 0.6079 - val_accuracy: 0.5910
Ep

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 24s - loss: 4.1555 - accuracy: 0.5893 - val_loss: 0.6806 - val_accuracy: 0.5915
Epoch 2/15
400/400 - 23s - loss: 0.6771 - accuracy: 0.5946 - val_loss: 0.6762 - val_accuracy: 0.5912
Epoch 3/15
400/400 - 23s - loss: 0.6721 - accuracy: 0.5946 - val_loss: 0.6697 - val_accuracy: 0.5919
Epoch 4/15
400/400 - 23s - loss: 0.6649 - accuracy: 0.5946 - val_loss: 0.6568 - val_accuracy: 0.5912
Epoch 5/15
400/400 - 23s - loss: 0.6504 - accuracy: 0.5947 - val_loss: 0.6406 - val_accuracy: 0.5915
Epoch 6/15
400/400 - 23s - loss: 0.6440 - accuracy: 0.5947 - val_loss: 0.6341 - val_accuracy: 0.5913
Epoch 7/15
400/400 - 23s - loss: 0.5870 - accuracy: 0.6505 - val_loss: 0.5873 - val_accuracy: 0.6265
Epoch 8/15
400/400 - 23s - loss: 0.5662 - accuracy: 0.6834 - val_loss: 0.5809 - val_accuracy: 0.6387
Epoch 9/15
400/400 - 23s - loss: 0.5656 - accuracy: 0.6849 - val_loss: 0.5704 - val_accuracy: 0.68

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 24s - loss: 14.5933 - accuracy: 0.6572 - val_loss: 0.8161 - val_accuracy: 0.7008
Epoch 2/15
400/400 - 23s - loss: 0.6221 - accuracy: 0.7334 - val_loss: 0.6529 - val_accuracy: 0.7219
Epoch 3/15
400/400 - 26s - loss: 0.4963 - accuracy: 0.7696 - val_loss: 0.6310 - val_accuracy: 0.7301
Epoch 4/15
400/400 - 23s - loss: 0.4482 - accuracy: 0.7909 - val_loss: 0.6375 - val_accuracy: 0.7326
Epoch 5/15
400/400 - 23s - loss: 0.4204 - accuracy: 0.8039 - val_loss: 0.6533 - val_accuracy: 0.7322
Epoch 6/15
400/400 - 23s - loss: 0.4031 - accuracy: 0.8119 - val_loss: 0.6801 - val_accuracy: 0.7307
Epoch 7/15
400/400 - 23s - loss: 0.3999 - accuracy: 0.8143 - val_loss: 0.7139 - val_accuracy: 0.7378
Epoch 8/15
400/400 - 23s - loss: 0.3871 - accuracy: 0.8209 - val_loss: 0.7506 - val_accuracy: 0.7399
Epoch 9/15
400/400 - 23s - loss: 0.3799 - accuracy: 0.8239 - val_loss: 0.8034 - val_accuracy: 0.7

...

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 55s - loss: 23.7293 - accuracy: 0.7191 - val_loss: 0.4886 - val_accuracy: 0.7769
Epoch 2/15
400/400 - 55s - loss: 0.4429 - accuracy: 0.7879 - val_loss: 0.4898 - val_accuracy: 0.7829
Epoch 3/15
400/400 - 47s - loss: 0.3880 - accuracy: 0.8158 - val_loss: 0.5069 - val_accuracy: 0.7804
Epoch 4/15
400/400 - 47s - loss: 0.3312 - accuracy: 0.8488 - val_loss: 0.5519 - val_accuracy: 0.7736
Epoch 5/15
400/400 - 47s - loss: 0.2827 - accuracy: 0.8778 - val_loss: 0.6150 - val_accuracy: 0.7609
Epoch 6/15
400/400 - 47s - loss: 0.2970 - accuracy: 0.8678 - val_loss: 0.7791 - val_accuracy: 0.7326
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 47s - loss: 37.0135 - accuracy: 0.7150 - val_loss: 0.5108 - val_accuracy: 0.7627
Epoch 2/15
400/400 - 47s - loss: 0.4837 - accuracy: 0.7717 - val_loss: 0.4833 - val_accuracy: 0.7699
Ep

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 25s - loss: 7.9754 - accuracy: 0.5899 - val_loss: 0.6334 - val_accuracy: 0.5913
Epoch 2/15
400/400 - 24s - loss: 0.6254 - accuracy: 0.5947 - val_loss: 0.6233 - val_accuracy: 0.5915
Epoch 3/15
400/400 - 24s - loss: 0.6015 - accuracy: 0.6088 - val_loss: 0.5943 - val_accuracy: 0.6425
Epoch 4/15
400/400 - 24s - loss: 0.5773 - accuracy: 0.6724 - val_loss: 0.5717 - val_accuracy: 0.6746
Epoch 5/15
400/400 - 24s - loss: 0.5600 - accuracy: 0.7017 - val_loss: 0.5625 - val_accuracy: 0.7115
Epoch 6/15
400/400 - 24s - loss: 0.5435 - accuracy: 0.7189 - val_loss: 0.5460 - val_accuracy: 0.7318
Epoch 7/15
400/400 - 24s - loss: 0.5327 - accuracy: 0.7308 - val_loss: 0.5398 - val_accuracy: 0.7375
Epoch 8/15
400/400 - 24s - loss: 0.5251 - accuracy: 0.7358 - val_loss: 0.5368 - val_accuracy: 0.7431
Epoch 9/15
400/400 - 24s - loss: 0.5164 - accuracy: 0.7406 - val_loss: 0.5383 - val_accuracy: 0.74

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 26s - loss: 6.4690 - accuracy: 0.5919 - val_loss: 0.6797 - val_accuracy: 0.5918
Epoch 2/15
400/400 - 25s - loss: 0.6767 - accuracy: 0.5946 - val_loss: 0.6763 - val_accuracy: 0.5919
Epoch 3/15
400/400 - 25s - loss: 0.6751 - accuracy: 0.5946 - val_loss: 0.6759 - val_accuracy: 0.5917
Epoch 4/15
400/400 - 25s - loss: 0.6745 - accuracy: 0.5946 - val_loss: 0.6748 - val_accuracy: 0.5916
Epoch 5/15
400/400 - 25s - loss: 0.6734 - accuracy: 0.5946 - val_loss: 0.6741 - val_accuracy: 0.5915
Epoch 6/15
400/400 - 25s - loss: 0.6714 - accuracy: 0.5946 - val_loss: 0.6711 - val_accuracy: 0.5921
Epoch 7/15
400/400 - 25s - loss: 0.6784 - accuracy: 0.5946 - val_loss: 0.6693 - val_accuracy: 0.5915
Epoch 8/15
400/400 - 25s - loss: 0.6612 - accuracy: 0.5945 - val_loss: 0.6493 - val_accuracy: 0.5921
Epoch 9/15
400/400 - 25s - loss: 0.6279 - accuracy: 0.6154 - val_loss: 0.6162 - val_accuracy: 0.61

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 68s - loss: 13.6574 - accuracy: 0.5932 - val_loss: 0.5993 - val_accuracy: 0.5921
Epoch 2/15
400/400 - 67s - loss: 0.5742 - accuracy: 0.6708 - val_loss: 0.5855 - val_accuracy: 0.7180
Epoch 3/15
400/400 - 67s - loss: 0.5648 - accuracy: 0.6994 - val_loss: 0.5638 - val_accuracy: 0.7106
Epoch 4/15
400/400 - 67s - loss: 0.5628 - accuracy: 0.6986 - val_loss: 0.5738 - val_accuracy: 0.7084
Epoch 5/15
400/400 - 67s - loss: 0.5604 - accuracy: 0.7017 - val_loss: 0.5763 - val_accuracy: 0.7029
Epoch 6/15
400/400 - 67s - loss: 0.5567 - accuracy: 0.7046 - val_loss: 0.5773 - val_accuracy: 0.7024
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 400 steps, validate for 171 steps
Epoch 1/15
400/400 - 67s - loss: 12.8547 - accuracy: 0.5928 - val_loss: 0.6424 - val_accuracy: 0.5915
Epoch 2/15
400/400 - 67s - loss: 0.6269 - accuracy: 0.5952 - val_loss: 0.6220 - val_accuracy: 0.5936
Ep

INFO:tensorflow:Oracle triggered exit


In [11]:
tuner.results_summary()